<a href="https://colab.research.google.com/github/Kaleab1999/Ethiopia-airlines-tweeter-review-sentiment-analysis/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install text-hammer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 5.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import text_hammer as th

/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [ ]:
import re
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/merge-csv.com__6361876ee50a8.csv', encoding = 'unicode_escape')
df.head()

,created_at,user_id,name,label,tweet,language,link,Column1
0,2022-10-28 23:15:24 UTC,4.386524e+08,AfricanSon®,negative,Ethiopian Airlines (EA) flight ET-3717 was int...,en,https://twitter.com/samsenge/status/1586134568...,NaN
1,2022-10-28 22:53:28 UTC,1.320000e+18,Bargo YOUSIF,negative,"_ details: It's #Ethiopian_Airlines flight, sm...",en,https://twitter.com/BargoYousif/status/1586129...,NaN
2,2022-10-28 22:21:31 UTC,8.226798e+08,Dr. Aklog Birara,positive,I look forward to seeing integrated infrastruc...,en,https://twitter.com/DrAklogBirara/status/15861...,NaN
3,2022-10-28 19:46:19 UTC,1.470000e+18,bill,negative,@jaeyoonieee me getting on ethiopian airlines ...,en,https://twitter.com/ysfkd_/status/158608195096...,NaN
4,2022-10-28 16:40:48 UTC,1.570000e+18,OverMeon,neutral,#ETH552 ET-AVE A350 941 Ethiopian Airlines ADD...,en,https://twitter.com/OverMeon/status/1586035262...,NaN


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def get_clean(x):
    doc = nlp(x)

    x = [ word.lemma_.lower().strip() for word in doc ]
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = th.cont_exp(x)
    x = th.remove_emails(x)
    x = th.remove_urls(x)
    x = th.remove_html_tags(x)
    x = th.remove_rt(x)
    x = th.remove_accented_chars(x)
    x = th.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    
    return x

In [ ]:
df['clean']= df['tweet'].apply(get_clean)
df.head()

,created_at,user_id,name,label,tweet,language,link,Column1,clean
0,2022-10-28 23:15:24 UTC,4.386524e+08,AfricanSon®,negative,Ethiopian Airlines (EA) flight ET-3717 was int...,en,https://twitter.com/samsenge/status/1586134568...,NaN,ethiopian airlines ea flight et3717 be interce...
1,2022-10-28 22:53:28 UTC,1.320000e+18,Bargo YOUSIF,negative,"_ details: It's #Ethiopian_Airlines flight, sm...",en,https://twitter.com/BargoYousif/status/1586129...,NaN,detail it be ethiopian airline flight smoke wh...
2,2022-10-28 22:21:31 UTC,8.226798e+08,Dr. Aklog Birara,positive,I look forward to seeing integrated infrastruc...,en,https://twitter.com/DrAklogBirara/status/15861...,NaN,i look forward to see integrate infrastructure...
3,2022-10-28 19:46:19 UTC,1.470000e+18,bill,negative,@jaeyoonieee me getting on ethiopian airlines ...,en,https://twitter.com/ysfkd_/status/158608195096...,NaN,jaeyoonie i get on ethiopian airline and be ov...
4,2022-10-28 16:40:48 UTC,1.570000e+18,OverMeon,neutral,#ETH552 ET-AVE A350 941 Ethiopian Airlines ADD...,en,https://twitter.com/OverMeon/status/1586035262...,NaN,eth552 et ave a350 941 ethiopian airlines add ...


In [ ]:
X= df['clean']
df['label']= df['label'].astype('category').cat.codes
y= df['label']

In [ ]:
y.shape

(4429,)

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=10000

One hot representation

In [ ]:
mes= X.copy()

In [ ]:
mes.head()

0    ethiopian airlines ea flight et3717 be interce...
1    detail it be ethiopian airline flight smoke wh...
2    i look forward to see integrate infrastructure...
3    jaeyoonie i get on ethiopian airline and be ov...
4    eth552 et ave a350 941 ethiopian airlines add ...
Name: clean, dtype: object

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in mes] 
onehot_repr[0]

[680,
 1170,
 1582,
 3439,
 1117,
 9378,
 215,
 7157,
 1820,
 5700,
 9478,
 1519,
 4838,
 847,
 4025,
 1528,
 9584,
 8602,
 693,
 8899,
 6216,
 28,
 8900,
 7090,
 4915,
 698,
 4528,
 6908,
 3942,
 2484]

In [ ]:
df['length']= df['clean'].apply(lambda x: len(x.split()))

df['length'].max(), df['length'].mean()

(68, 20.237525400767666)

In [ ]:
sent_length=80
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[ 680 1170 1582 ...    0    0    0]
 [2366 9275 9378 ...    0    0    0]
 [5365 9116 1662 ...    0    0    0]
 ...
 [1535  680 5820 ...    0    0    0]
 [ 680 1170 9378 ...    0    0    0]
 [1535  680 5942 ...    0    0    0]]


In [ ]:
embedded_docs[0]

array([ 680, 1170, 1582, 3439, 1117, 9378,  215, 7157, 1820, 5700, 9478,
       1519, 4838,  847, 4025, 1528, 9584, 8602,  693, 8899, 6216,   28,
       8900, 7090, 4915,  698, 4528, 6908, 3942, 2484,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0], dtype=int32)

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((4429, 80), (4429,))

In [ ]:
y_final

array([0, 0, 2, ..., 2, 2, 2], dtype=int8)

In [ ]:
df['label']

0       0
1       0
2       2
3       0
4       1
       ..
4424    1
4425    1
4426    1
4427    1
4428    1
Name: label, Length: 4429, dtype: int8

In [ ]:
from imblearn.over_sampling import SMOTE

over= SMOTE()

X_final, y_final= over.fit_resample(X_final, y_final)

In [ ]:
X_final.shape,y_final.shape

((6822, 80), (6822,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42, stratify= y_final)

In [ ]:
## Creating model
embedding_vector_features=70
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100)),
tf.keras.layers.Dense(embedding_vector_features, activation='relu'),
model.add(Dense(1,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 80, 70)            700000    
                                                                 
 lstm_15 (LSTM)              (None, 100)               68400     
                                                                 
 dense_27 (Dense)            (None, 1)                 101       
                                                                 
Total params: 768,501
Trainable params: 768,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=100)

Epoch 1/5
55/55 [==============================] - 14s 213ms/step - loss: 0.0000e+00 - accuracy: 0.3333 - val_loss: 0.0000e+00 - val_accuracy: 0.3333
Epoch 2/5
55/55 [==============================] - 11s 205ms/step - loss: 0.0000e+00 - accuracy: 0.3333 - val_loss: 0.0000e+00 - val_accuracy: 0.3333
Epoch 3/5
55/55 [==============================] - 11s 205ms/step - loss: 0.0000e+00 - accuracy: 0.3333 - val_loss: 0.0000e+00 - val_accuracy: 0.3333
Epoch 4/5
55/55 [==============================] - 11s 206ms/step - loss: 0.0000e+00 - accuracy: 0.3333 - val_loss: 0.0000e+00 - val_accuracy: 0.3333
Epoch 5/5
55/55 [==============================] - 11s 205ms/step - loss: 0.0000e+00 - accuracy: 0.3333 - val_loss: 0.0000e+00 - val_accuracy: 0.3333


adding droupout

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=80
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
y_pred= model.predict(X_test)

43/43 [==============================] - 1s 20ms/step


In [ ]:
model1 = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(voc_size, embedding_vector_features),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_vector_features)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_vector_features, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(3, activation='softmax')
])
model1.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, None, 70)          700000    
                                                                 
 bidirectional_7 (Bidirectio  (None, 140)              78960     
 nal)                                                            
                                                                 
 dense_24 (Dense)            (None, 70)                9870      
                                                                 
 dense_25 (Dense)            (None, 3)                 213       
                                                                 
Total params: 789,043
Trainable params: 789,043
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 20
batch=50
history = model1.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), verbose=2, batch_size=batch)

Epoch 1/20
110/110 - 55s - loss: 0.8511 - accuracy: 0.5881 - val_loss: 0.7288 - val_accuracy: 0.6645 - 55s/epoch - 498ms/step
Epoch 2/20
110/110 - 13s - loss: 0.5214 - accuracy: 0.7955 - val_loss: 0.7380 - val_accuracy: 0.6894 - 13s/epoch - 119ms/step
Epoch 3/20
110/110 - 13s - loss: 0.2673 - accuracy: 0.9100 - val_loss: 0.8841 - val_accuracy: 0.6996 - 13s/epoch - 120ms/step
Epoch 4/20
110/110 - 13s - loss: 0.1352 - accuracy: 0.9560 - val_loss: 1.1246 - val_accuracy: 0.7055 - 13s/epoch - 120ms/step
Epoch 5/20
110/110 - 13s - loss: 0.0892 - accuracy: 0.9738 - val_loss: 1.2968 - val_accuracy: 0.6974 - 13s/epoch - 119ms/step
Epoch 6/20
110/110 - 13s - loss: 0.0563 - accuracy: 0.9848 - val_loss: 1.5169 - val_accuracy: 0.6989 - 13s/epoch - 119ms/step
Epoch 7/20
110/110 - 13s - loss: 0.0427 - accuracy: 0.9886 - val_loss: 1.7630 - val_accuracy: 0.7040 - 13s/epoch - 118ms/step
Epoch 8/20
110/110 - 13s - loss: 0.0412 - accuracy: 0.9888 - val_loss: 1.5529 - val_accuracy: 0.6996 - 13s/epoch - 119